In [2]:
import cv2 

# Load the image 
#img = cv2.imread('r:C:\Users\Shachi\OneDrive\Pictures\Camera Roll\Shachi Photo jee.jpg')
img = cv2.imread("C:\\Users\\Shachi\\OneDrive\\Pictures\\Camera Roll\\Shachi Photo jee.jpg")

# Convert to grayscale 
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

#to separate the object from the background 
ret, thresh = cv2.threshold(gray, 127, 255, 0) 

# Find the contours of the object 
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

# Draw the contours on the original image 
cv2.drawContours(img, contours, -1, (0,255,0), 3) 

# Get the area of the object in pixels 
area = cv2.contourArea(contours[0]) 

# Convert the area from pixels to a real-world unit of measurement (e.g. cm^2) 
scale_factor = 0.1 # 1 pixel = 0.1 cm 
size = area * scale_factor ** 2

# Print the size of the object 
print('Size:', size) 

# Display the image with the contours drawn 
cv2.imwrite('Object.jpeg', img) 
cv2.waitKey(0) 

# Save the image with the contours drawn to a file 
cv2.imwrite('object_with_contours.jpg', img)


Size: 0.28500000000000003


True

In [2]:
pip install opencv-python numpy dlib imutils

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached imutils-0.5.4-py3-none-any.whl
  Running setup.py clean for dlib
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [76 lines of output]
      running bdist_wheel
      running build
      running build_ext
      C:\Users\Shachi\AppData\Local\Temp\pip-install-ryin7e69\dlib_108c108c26ba43b48d026f1686df6319\setup.py:163: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        if LooseVersion(cmake_version) < '3.1.0':
      Building extension for Python 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
      Invoking CMake setup: 'cmake C:\Users\Shachi\AppData\Local\Temp\pip-install-ryin7e69\dlib_108c108c26ba43b48d026f1686df6319\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\Shachi\AppData\Local\Temp\pip-install-ryin7e69\dlib_108c108c26ba43b48d026f1686df6319\build\lib.win-amd64-cpython-311 -DPYTHON_EXECUTABLE=C:\Users\Shachi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz

In [4]:


import urllib.request
import bz2
import shutil

url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
filename = "shape_predictor_68_face_landmarks.dat.bz2"

print("Downloading model file...")
urllib.request.urlretrieve(url, filename)

print("Extracting file...")
with bz2.BZ2File(filename) as fr, open("shape_predictor_68_face_landmarks.dat", "wb") as fw:
    shutil.copyfileobj(fr, fw)

print("Download and extraction complete!")


Extracting file...
Download and extraction complete!


In [10]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

# Define eye landmarks
LEFT_EYE = list(range(42, 48))
RIGHT_EYE = list(range(36, 42))

EAR_THRESHOLD = 0.25  # Threshold below which eyes are considered closed
BLINK_FRAMES = 3       # Number of frames to detect a blink

drowsy_counter = 0
blink_counter = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
        
        left_eye = landmarks[LEFT_EYE]
        right_eye = landmarks[RIGHT_EYE]
        
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0
        
        if ear < EAR_THRESHOLD:
            drowsy_counter += 1
        else:
            if 1 <= drowsy_counter <= BLINK_FRAMES:
                blink_counter += 1
                cv2.putText(frame, f"Blinks: {blink_counter}", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            drowsy_counter = 0
        
        for point in np.concatenate((left_eye, right_eye), axis=0):
            cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
    
    cv2.imshow("Blink Detector", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 